In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import datasets
from multiprocessing.dummy import Pool
from datetime import datetime

# Pre-processing

In [2]:
covid = pd.read_csv('covid_test.csv')

C:\Users\malu.maia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,7,8,9,15,20,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,50,51,52,53,54,55,56,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
covid = covid[['estadoCaso', 'municipioCaso', 'bairroCaso', 'sexoCaso', 'dataNascimento', 
               'idadeCaso','racaCor', 'resultadoFinalExame']]
global df
df = covid[:1000]

In [4]:
# joining the three locality attributes to construct a generalization tree
local = []
for i in range(len(df)):
    local.append('{}, {}, {}'.format(df['bairroCaso'].iloc[i], df['municipioCaso'].iloc[i], df['estadoCaso'].iloc[i]))
    
df = df.drop(['estadoCaso', 'municipioCaso', 'bairroCaso'], axis=1)
df['localCaso'] = local
df = df[['localCaso', 'sexoCaso', 'dataNascimento', 'idadeCaso', 'racaCor', 'resultadoFinalExame']]
# changing date format
df = df.copy()
for i in range(len(df)):
    df['dataNascimento'].iloc[i] = '{}/{}/{}'.format(df['dataNascimento'].iloc[i][-2:], 
                                                     df['dataNascimento'].iloc[i][5:7],
                                                     df['dataNascimento'].iloc[i][:4])
df.head()

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"CAICARA, SOBRAL, CE",MASCULINO,14/08/2003,17.0,Parda,Negativo
1,"BANGUE, PACAJUS, CE",MASCULINO,07/11/1983,37.0,Parda,Negativo
2,"ZUMBI, FORTALEZA, CE",MASCULINO,12/03/1992,28.0,Parda,Negativo
3,"METROPOLIS, CAUCAIA, CE",MASCULINO,06/03/1970,50.0,Sem Informacao,Negativo
4,"PARANGABA, FORTALEZA, CE",FEMININO,10/01/1939,81.0,Parda,Negativo


# Suppression

In [5]:
def suppression_value(feature, value=''):
    df_s = df.copy()
    if(value == ''):
        for line in range(len(df_s)):
            df_s[feature].iloc[line] = '*'
    else:
        df_s = df_s.replace(value, '*')
    return df_s

In [6]:
def suppression(features):
    df_s = df.copy()
    for feature in features:
        values = np.unique(df[feature])
        dict_rep = {key: '*' for key in values}
        df_s[feature] = df_s[feature].replace(dict_rep)
    return df_s

### Testing Suppression on COVID data

In [7]:
suppression(['racaCor']).head()

,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"CAICARA, SOBRAL, CE",MASCULINO,14/08/2003,17.0,*,Negativo
1,"BANGUE, PACAJUS, CE",MASCULINO,07/11/1983,37.0,*,Negativo
2,"ZUMBI, FORTALEZA, CE",MASCULINO,12/03/1992,28.0,*,Negativo
3,"METROPOLIS, CAUCAIA, CE",MASCULINO,06/03/1970,50.0,*,Negativo
4,"PARANGABA, FORTALEZA, CE",FEMININO,10/01/1939,81.0,*,Negativo


In [8]:
suppression_value('racaCor', 'Parda').head()

,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"CAICARA, SOBRAL, CE",MASCULINO,14/08/2003,17.0,*,Negativo
1,"BANGUE, PACAJUS, CE",MASCULINO,07/11/1983,37.0,*,Negativo
2,"ZUMBI, FORTALEZA, CE",MASCULINO,12/03/1992,28.0,*,Negativo
3,"METROPOLIS, CAUCAIA, CE",MASCULINO,06/03/1970,50.0,Sem Informacao,Negativo
4,"PARANGABA, FORTALEZA, CE",FEMININO,10/01/1939,81.0,*,Negativo


# Generalization

In [9]:
def generalization(level_local=0, level_bday=0):
    df_g = df.copy()
    df_g = generalizing_local(level_local)
    df_g = generalizing_bday(df_g, level_bday)
    
    return df_g

### Generalizing County

In [11]:
def generalizing_local(level=0):
    df_g = df.copy()
    if(level == 1):
        for i in range(len(df_g)):
            neigh, county, state = df_g['localCaso'].iloc[i].split(', ')
            df_g['localCaso'].iloc[i] = '*, {}, {}'.format(county, state)
    elif(level == 2):
        for i in range(len(df)):
            neigh, county, state = df_g['localCaso'].iloc[i].split(', ')
            df_g['localCaso'].iloc[i] = '*, *, {}'.format(state)
    
    return df_g

In [12]:
generalizing_local(2)

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"*, *, CE",MASCULINO,14/08/2003,17.0,Parda,Negativo
1,"*, *, CE",MASCULINO,07/11/1983,37.0,Parda,Negativo
2,"*, *, CE",MASCULINO,12/03/1992,28.0,Parda,Negativo
3,"*, *, CE",MASCULINO,06/03/1970,50.0,Sem Informacao,Negativo
4,"*, *, CE",FEMININO,10/01/1939,81.0,Parda,Negativo
...,...,...,...,...,...,...
995,"*, *, CE",MASCULINO,27/12/1989,30.0,Parda,Positivo
996,"*, *, CE",FEMININO,21/07/1988,32.0,Parda,Negativo
997,"*, *, CE",FEMININO,06/01/1928,92.0,Branca,Negativo
998,"*, *, CE",FEMININO,18/05/1983,37.0,Branca,Negativo


### Generalizing birthday

In [13]:
def generalizing_bday(df_g, level=0):
#    df_g = df.copy()
    if(level == 1):
        for i in range(len(df_g)):
            day, month, year = df_g['dataNascimento'].iloc[i].split('/')
            df_g['dataNascimento'].iloc[i] = '*/{}/{}'.format(month, year)
    elif(level == 2):
        for i in range(len(df)):
            day, month, year = df_g['dataNascimento'].iloc[i].split('/')
            df_g['dataNascimento'].iloc[i] = '*/*/{}'.format(year)
    
    return df_g

In [14]:
generalizing_bday(df, 1)

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"CAICARA, SOBRAL, CE",MASCULINO,*/08/2003,17.0,Parda,Negativo
1,"BANGUE, PACAJUS, CE",MASCULINO,*/11/1983,37.0,Parda,Negativo
2,"ZUMBI, FORTALEZA, CE",MASCULINO,*/03/1992,28.0,Parda,Negativo
3,"METROPOLIS, CAUCAIA, CE",MASCULINO,*/03/1970,50.0,Sem Informacao,Negativo
4,"PARANGABA, FORTALEZA, CE",FEMININO,*/01/1939,81.0,Parda,Negativo
...,...,...,...,...,...,...
995,"PRESIDENTE KENNEDY, FORTALEZA, CE",MASCULINO,*/12/1989,30.0,Parda,Positivo
996,"JARDIM CEARENSE, FORTALEZA, CE",FEMININO,*/07/1988,32.0,Parda,Negativo
997,"CENTRO, ICO, CE",FEMININO,*/01/1928,92.0,Branca,Negativo
998,"ALDEITA, FORTALEZA, CE",FEMININO,*/05/1983,37.0,Branca,Negativo


In [15]:
generalization(1, 2)

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"*, SOBRAL, CE",MASCULINO,*/*/2003,17.0,Parda,Negativo
1,"*, PACAJUS, CE",MASCULINO,*/*/1983,37.0,Parda,Negativo
2,"*, FORTALEZA, CE",MASCULINO,*/*/1992,28.0,Parda,Negativo
3,"*, CAUCAIA, CE",MASCULINO,*/*/1970,50.0,Sem Informacao,Negativo
4,"*, FORTALEZA, CE",FEMININO,*/*/1939,81.0,Parda,Negativo
...,...,...,...,...,...,...
995,"*, FORTALEZA, CE",MASCULINO,*/*/1989,30.0,Parda,Positivo
996,"*, FORTALEZA, CE",FEMININO,*/*/1988,32.0,Parda,Negativo
997,"*, ICO, CE",FEMININO,*/*/1928,92.0,Branca,Negativo
998,"*, FORTALEZA, CE",FEMININO,*/*/1983,37.0,Branca,Negativo


# Perturbation

In [16]:
from scipy.stats import laplace

In [17]:
def adding_laplace_noise(index, value):
    noise = laplace.rvs(loc=0, scale = sensitivity/eps)
    value = np.ceil(round(value + noise, 1)) # remove ceil(*) for float values
   
    return index, value

In [18]:
def perturbation(feature):
    global sensitivity
    global eps
    df_copy = df.copy()[:20000]
    if(isinstance(df_copy[feature].iloc[0], str)):
        values, counts = df_copy[feature].value_counts().index, df_copy[feature].value_counts().values
        df_copy[feature] = random.choices(values, weights=counts, k=len(df_copy[feature])) # following the real frequencies distribution
    else:
        sensitivity = max(df_copy[feature]) - min(df_copy[feature])
        eps = 10
        index_feature_2D = np.c_[np.array(df_copy.index), np.array(df_copy[feature])]
        
        with Pool(15) as pool:
            threads = pool.starmap(adding_laplace_noise, index_feature_2D)
            pool.terminate()
            pool.close()
        pool.join()
        
        df_copy[feature] = [value for value in list(dict(threads).values())]
    
    return df_copy

### Testing Perturbation on COVID data

In [19]:
feature = 'idadeCaso'
df_p = perturbation(feature)
df_p

,localCaso,sexoCaso,dataNascimento,idadeCaso,racaCor,resultadoFinalExame
0,"CAICARA, SOBRAL, CE",MASCULINO,*/08/2003,18.0,Parda,Negativo
1,"BANGUE, PACAJUS, CE",MASCULINO,*/11/1983,47.0,Parda,Negativo
2,"ZUMBI, FORTALEZA, CE",MASCULINO,*/03/1992,27.0,Parda,Negativo
3,"METROPOLIS, CAUCAIA, CE",MASCULINO,*/03/1970,56.0,Sem Informacao,Negativo
4,"PARANGABA, FORTALEZA, CE",FEMININO,*/01/1939,90.0,Parda,Negativo
...,...,...,...,...,...,...
995,"PRESIDENTE KENNEDY, FORTALEZA, CE",MASCULINO,*/12/1989,37.0,Parda,Positivo
996,"JARDIM CEARENSE, FORTALEZA, CE",FEMININO,*/07/1988,24.0,Parda,Negativo
997,"CENTRO, ICO, CE",FEMININO,*/01/1928,84.0,Branca,Negativo
998,"ALDEITA, FORTALEZA, CE",FEMININO,*/05/1983,61.0,Branca,Negativo


In [20]:
print('Médias perturbada e não-perturbada: \n- {}\n- {}\n'.format(np.mean(df_p[feature]), 
                                                                np.mean(df[feature].iloc[:20000])))
print('Medianas perturbada e não-perturbada: \n- {}\n- {}\n'.format(np.median(df_p[feature]), 
                                                                  np.median(df[feature].iloc[:20000])))

Médias perturbada e não-perturbada: 
- 39.611
- 38.97

Medianas perturbada e não-perturbada: 
- 38.0
- 35.0

